In [196]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [355]:
file_1 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/processed_prices_ubereats_nonca_fullserv_03252024.csv"
file_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/final_processed_prices_ubereats_nonca_fullserv_05162024.csv"
pre = pd.read_csv(file_1)
post = pd.read_csv(file_2)

file_path_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/uszips.csv"
ca_zip_count = pd.read_csv(file_path_2)

In [356]:
# combined = pd.concat([pre,post])
# combined.to_csv("processed_prices_ubereats_ca_smallff_03262024.csv")

In [357]:
common_addresses = pd.merge(pre[['restaurant_location']], post[['restaurant_location']], on='restaurant_location', how='inner')

# Filter both DataFrames to include only the matched addresses
pre_filtered = pre[pre['restaurant_location'].isin(common_addresses['restaurant_location'])]
post_filtered = post[post['restaurant_location'].isin(common_addresses['restaurant_location'])]


In [358]:
def mean_non_zero(x):
    return np.mean(x[x != 0]) if np.any(x != 0) else 0

def median_non_zero(x):
    return np.median(x[x != 0]) if np.any(x != 0) else 0

def std_non_zero(x):
    return np.std(x[x != 0]) if np.any(x != 0) else 0

In [359]:
post_filtered.drop(post_filtered[post_filtered['restaurant_name'] == 'Panera Bread'].index, inplace=True)


/var/folders/p9/_x396g015hb1f7h6k_mytrnc0000gn/T/ipykernel_52927/1055073687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_filtered.drop(post_filtered[post_filtered['restaurant_name'] == 'Panera Bread'].index, inplace=True)


In [360]:
#Unique Restaurants
len(post_filtered['restaurant_name'].unique())

6

In [361]:
post_filtered['restaurant_name'].unique()

array(['Outback Steakhouse', 'Buffalo Wild Wings', 'Applebee', 'Denny',
       'Red Robin', 'TGI Fridays'], dtype=object)

In [362]:
#Unique Counties 
len(post_filtered['county_name'].unique())

122

In [363]:
len(post_filtered['restaurant_location'].unique())

692

In [365]:
mean_non_zero(post_filtered['cheeseburger'])

12.533927536231829

In [180]:
combined.to_csv("final_processed_ubereats_ca_smallff_05272024")

sanity check

In [64]:
ca_ff['restaurant_location'] = ca_ff['restaurant_location'].astype('string')
ca_ff['restaurant_location'] = ca_ff['restaurant_location'].str.lower()

In [65]:
pattern = r",\s*([a-zA-Z]{2})\s*,?\s*(\d{5}(?:-\d{4})?)"

# Function to extract state and zip code from an address string
def extract_state_zip(address):
    match = re.search(pattern, address)
    if match:
        state, zip_code = match.groups()
        return state, zip_code
    else:
        return None, None

# Assuming ca_ff is your DataFrame
# Ensure the 'restaurant_location' column is of string type
ca_ff['restaurant_location'] = ca_ff['restaurant_location'].astype(str)

# Apply the function to extract state and zip code
ca_ff[['state', 'zip']] = ca_ff['restaurant_location'].apply(lambda x: pd.Series(extract_state_zip(x)))

# Convert 'zip' column to integer, handling potential None values
ca_ff['zip'] = ca_ff['zip'].str.split('-').str[0].astype('Int64')

# Assuming ca_zip_count is your DataFrame with columns 'zip' and 'county'
# Ensure the 'zip' column in ca_zip_count is also of integer type
ca_zip_count['zip'] = ca_zip_count['zip'].astype('Int64')

ca_zip_count = ca_zip_count[['zip', 'county_name']]

# Merge ca_ff with ca_zip_count on 'zip' column to get the county
ca_ff = ca_ff.merge(ca_zip_count, on='zip', how='left')

# Check the result
print(ca_ff.head())

   Unnamed: 0 restaurant_name                  menu_item  menu_item_price  \
0           0        McDonald        Medium French Fries             4.69   
1           1        McDonald  10 pc. Chicken McNuggets®             6.69   
2           2        McDonald                   Big Mac®             6.79   
3           3        McDonald        2 Cheeseburger Meal            10.99   
4           4        McDonald               Cheeseburger             3.49   

                       restaurant_location  \
0  1330 jackson st, oakland, ca, 94612, us   
1  1330 jackson st, oakland, ca, 94612, us   
2  1330 jackson st, oakland, ca, 94612, us   
3  1330 jackson st, oakland, ca, 94612, us   
4  1330 jackson st, oakland, ca, 94612, us   

                           inputted_location restaurant_rating  \
0  1 Frank H. Ogawa Plaza, Oakland, CA 94612              4.6    
1  1 Frank H. Ogawa Plaza, Oakland, CA 94612              4.6    
2  1 Frank H. Ogawa Plaza, Oakland, CA 94612              4.6 

In [93]:
ca_ff['county_name'].unique()

array(['San Bernardino', 'Los Angeles', 'Kern', 'Sacramento',
       'San Joaquin', 'Solano', 'Orange', 'Sonoma', 'Santa Clara',
       'San Francisco', 'San Mateo', 'San Diego', 'Santa Barbara',
       'Contra Costa', 'Placer', 'Santa Cruz', 'Monterey', 'Riverside',
       'Fresno', 'Stanislaus', 'Alameda', 'Tulare', 'San Luis Obispo',
       'Yolo', 'Ventura', 'Lake', 'Wake', 'Marin', 'Lancaster',
       'New Hanover', 'Orleans', 'Hidalgo', 'Rutherford', 'Lehigh',
       'Greenville', 'Berks', 'Tarrant', 'Mecklenburg', 'Napa', 'Madera',
       'Kings', 'El Dorado'], dtype=object)

In [16]:
ca_ff[['hamburger', 'cheeseburger', 'specialty_item', 'restaurant_name']]

,hamburger,cheeseburger,specialty_item,restaurant_name
0,3.69,4.59,8.19,McDonald
1,3.36,3.74,6.86,Jack in the Box
2,5.39,3.49,10.39,Burger King
3,NaN,5.74,8.74,Carls Jr
4,3.89,4.39,8.29,McDonald
...,...,...,...,...
1315,11.15,12.47,15.47,Five Guys
1316,9.59,10.79,13.67,Five Guys
1317,10.19,11.39,14.39,Five Guys
1318,11.15,12.47,15.47,Five Guys


In [20]:
mean_non_zero(ca_ff['specialty_item'])

8.86307283763271

In [121]:
agg_dict = {
    'menu_item_price mean_non_zero': mean_non_zero,
    'menu_item_price median_non_zero': mean_non_zero,
    'menu_item_price std_non_zero': mean_non_zero,
    'restaurant_rating mean': mean_non_zero,
    'menu_item count': mean_non_zero,
    'hamburger': mean_non_zero,
    'cheeseburger': mean_non_zero,
    'specialty_item': mean_non_zero,
    'fries': mean_non_zero,
    'combo': mean_non_zero,
    'county_name': 'count'
}

# Perform the groupby and aggregation
result = ca_ff.groupby("restaurant_name").agg(agg_dict)

# Reset index to turn 'restaurant_name' from index to a column
result = result.reset_index()

# Rename the columns to more readable names
result.columns = [
    'Restaurant Name', 
    'Average Menu Item Price', 
    'Median Menu Item Price', 
    'Std Dev Menu Item Price', 
    'Average Restaurant Rating', 
    'Average Menu Item Count', 
    'Average Hamburger Price', 
    'Average Cheeseburger Price', 
    'Average Specialty Item Price', 
    'Average Fries Price', 
    'Average Combo Price', 
    'County Count'
]

result

,Restaurant Name,Average Menu Item Price,Median Menu Item Price,Std Dev Menu Item Price,Average Restaurant Rating,Average Menu Item Count,Average Hamburger Price,Average Cheeseburger Price,Average Specialty Item Price,Average Fries Price,Average Combo Price,County Count
0,Applebee,20.593280,15.296237,20.962820,4.273656,140.387097,13.808602,14.706720,NaN,14.875920,NaN,186
1,Buffalo Wild Wings,7.729876,6.330000,8.967980,4.193636,109.500000,NaN,13.498182,NaN,4.867315,NaN,110
2,Denny,10.023410,8.517061,6.086510,4.223030,147.981818,NaN,13.658405,NaN,5.215333,NaN,165
3,Outback Steakhouse,18.067444,14.071364,12.957996,4.385714,109.324675,14.948533,10.470519,NaN,5.064026,NaN,77
4,Panera Bread,8.875863,7.863333,7.724865,4.653333,184.666667,NaN,NaN,NaN,NaN,NaN,15
5,Red Robin,12.050759,11.203814,7.822556,4.467797,130.644068,NaN,8.345254,NaN,4.543158,NaN,59
6,TGI Fridays,25.089391,16.582069,26.598320,4.179310,179.068966,NaN,13.705862,NaN,3.167931,NaN,29
